# Organización de número de empleados por sector

## Limpieza de los datos

<b>Objetivo:</b> eliminar las líneas innecesarias, y organizar los encabezados de los documentos. De esta manera podremos unificar todos los datos posteriormente.

In [18]:
import os

def clean_data(input_folder, output_folder, year):

    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['Serie;;;2015;2016;2017;2018;2019;2020;2021;2022;2023(A)', 'empresarial']
    headers_2023 = 'Serie;Codigo_municipio;Nombre;2022;2023\n'
    headers_2020 = 'Serie;Codigo_municipio;Nombre;2020\n'

    # Limpiar y guardar los archivos CSV en la carpeta de salida
    for file in csv_files:
        original_file = os.path.join(input_folder, file)
        clean_file = os.path.join(output_folder, file)
        with open(original_file, 'r') as f_input, open(clean_file, 'w') as f_output:
            if year == 2020:
                f_output.write(headers_2020)
            else:
                f_output.write(headers_2023)
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line or line.strip() == ';':
                            delete = True
                    if line.strip() == '':
                        delete = True
                    elif '(A) Avance' in line:
                        line = line.replace('(A) Avance ;', '')
                    if not delete:
                        fields = line.strip().split(';')
                        if year == 2023:
                            f_output.write(';'.join(fields[:3] + fields[-3:]) + '\n')
                        else:
                            f_output.write(';'.join(fields[:3] + fields[8:9]) + '\n')

                    # Eliminar líneas posteriores al ser el último municipio
                    if 'Zarzalejo' in line:
                        delete_next = True

clean_data(".\original_data", ".\clean_data", 2023)
clean_data(".\original_data", ".\clean_data_2020", 2020)


## Suma y división por sectores

<b>Objetivo:</b> dividir los archivos de la carpeta por sectores, y realizar las sumas parciales del número de empleos por sector

In [1]:
import os, csv

def sector_divider(input_folder, output_folder, year):

    # Diccionario para almacenar los datos sumados por sector
    sector_data = {'primario': {}, 'secundario': {}, 'terciario': {}}

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    csv_files = os.listdir(input_folder)

    # Mapear cada archivo a su sector correspondiente y sumar los datos
    for file in csv_files:
        if file.endswith('.csv'):

            # Determinar el sector según el nombre del archivo
            if 'agricultura' in file or 'ganaderia' in file or 'pesca' in file:
                sector = 'primario'
            elif 'industria' in file or 'construccion' in file or 'electricidad' in file or 'agua' in file or 'metalurgia' in file:
                sector = 'secundario'
            else:
                sector = 'terciario'

            # Leer el archivo y sumar los datos por sector
            with open(os.path.join(input_folder, file), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                next(csv_reader)

                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""

                    # Verificar si ya existe una entrada para esta serie y código de municipio
                    key = (serie, codigo_municipio, nombre)
                    if key in sector_data[sector]:
                        # Sumar los valores de cada año
                        for i in range(3, len(row)):
                            sector_data[sector][key][i] += int(row[i]) if row[i] else 0
                    else:
                        # Si no existe, crear una nueva entrada en el diccionario
                        sector_data[sector][key] = [serie, codigo_municipio, nombre] + [int(x) if x else 0 for x in row[3:]]

    # Escribir los datos sumados por sector en archivos CSV
    for sector, datos_sumados in sector_data.items():
        output_path = os.path.join(output_folder, f'suma_{sector}.csv')

        with open(output_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file, delimiter=';')
            if year == 2023:
                csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', '2022', '2023'])
            else:
                csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', '2020'])
            for key, values in datos_sumados.items():
                csv_writer.writerow([key[0], key[1], key[2], *values[3:5]])

sector_divider("./clean_data", "./suma_ocupados_por_sector_2023", 2023)
sector_divider("./clean_data_2020", "./suma_ocupados_por_sector_2020", 2020)

## Unión de todos los datos de empleo por sector

<b>Objetivo:</b> unir todos los archivos de la carpeta en uno solo, con sus encabezados correspondientes. Guardar el <i>.csv</i> resultante en la carpeta <i>union_final</i> para poder añadir los datos al archivo final.

In [20]:
import os, csv

def join_data(folder_path, output_path, year):
    # Crear un diccionario para almacenar los datos sumados
    join_data = {}

    # Obtener la lista de files en la carpeta
    csv_file = os.listdir(folder_path)

    # Iterar sobre cada archivo en la carpeta
    for file in csv_file:
        if file.endswith('.csv'):
            with open(os.path.join(folder_path, file), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                # Saltar la primera fila que contiene los headers
                next(csv_reader)

                # Obtener el sector del nombre del archivo
                sector = file.split('.')[0].split('_')[1]

                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""

                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    if (serie, codigo_municipio, nombre) not in join_data:
                        join_data[(serie, codigo_municipio, nombre)] = {}

                    if year == 2023:
                        # Obtener los datos para cada intervalo de edad
                        for i, value in enumerate(row[3:5], start=2022):
                            key = f"ocupados_{sector}_{i}"
                            # Agregar los datos al diccionario
                            join_data[(serie, codigo_municipio, nombre)][key] = int(value) if value else 0
                    else:
                        # Obtener los datos para cada intervalo de edad
                        for i, value in enumerate(row[3:], start=2020):
                            key = f"ocupados_{sector}"
                            # Agregar los datos al diccionario
                            join_data[(serie, codigo_municipio, nombre)][key] = int(value) if value else 0

    # Escribir los datos sumados en un nuevo file CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')

        final_keys = []
        for key, values in join_data.items():
            if final_keys == []:
                final_keys.extend(values.keys())
        final_keys = [str(clave) for clave in final_keys]
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', *final_keys])

        # Escribir los datos sumados
        for key, values in join_data.items():
            fila = [key[0], key[1], key[2]]
            for clave in final_keys:
                fila.append(values.get(clave, ''))
            csv_writer.writerow(fila)

join_data(f'./suma_ocupados_por_sector_2023', '../union_final/data/empleo_sector.csv', 2023)
join_data(f'./suma_ocupados_por_sector_2020', '../union_2020/empleo_sector.csv', 2020)


In [21]:
import csv

def total_ocupados(input, output):
    # Abrir el archivo de entrada en modo lectura
    with open(input, 'r', newline='') as file_in:
        # Crear un lector CSV
        reader = csv.reader(file_in, delimiter=';')
        
        # Crear una lista para almacenar las filas filtradas
        filas_filtradas = []
        
        # Leer la cabecera y almacenarla en la lista de filas filtradas
        cabecera = next(reader)
        filas_filtradas.append(cabecera[:3] + ['ocupados_total_2023'])
        
        # Leer el resto de las filas y filtrarlas
        for fila in reader:
            # Obtener los valores necesarios
            serie, codigo_municipio, nombre, ocupados_total_2023 = fila[0], fila[1], fila[2], fila[4]
            # Almacenar en la lista de filas filtradas
            filas_filtradas.append([serie, codigo_municipio, nombre, ocupados_total_2023])
    
    # Escribir las filas filtradas en el archivo de salida
    with open(output, 'w', newline='') as file_out:
        # Crear un escritor CSV
        writer = csv.writer(file_out, delimiter=';')
        # Escribir las filas filtradas en el archivo de salida
        writer.writerows(filas_filtradas)

# Llamar a la función para exportar los datos
total_ocupados('./clean_data/total_ocupados.csv', '../union_final/data/empleo_total.csv')
